<a href="https://colab.research.google.com/github/harishkulkarni10/Credit-Card-Fraud-Detection/blob/development/notebooks/03_model_experiments.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
from google.colab import drive
drive.mount('/content/drive')

import joblib
import torch
import numpy as np
import pandas as pd

# Paths
model_path = "/content/drive/MyDrive/Data Science course/Major Projects/Projects/Credit Card Fraud Detection/Credit Card Fraud Detection/models/BEST_XGBOOST_MODEL.pkl"
scaler_path = "/content/drive/MyDrive/Data Science course/Major Projects/Projects/Credit Card Fraud Detection/Credit Card Fraud Detection/data/scaler.pkl"
scaled_tensors_path = "/content/drive/MyDrive/Data Science course/Major Projects/Projects/Credit Card Fraud Detection/Credit Card Fraud Detection/data/scaled_tensors.pth"

# Load
best_model_data = joblib.load(model_path)
bst = best_model_data['model']
feature_cols = best_model_data['feature_names']
scaler = joblib.load(scaler_path)

print("Best XGBoost model loaded!")
print(f"AUC: {best_model_data['auc']:.4f} | Recall: {best_model_data['recall']:.4f}")
print(f"Features used: {len(feature_cols)}")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Best XGBoost model loaded!
AUC: 0.9770 | Recall: 0.8000
Features used: 30


In [5]:
def predict_fraud(raw_transaction_dict):

    df = pd.DataFrame([raw_transaction_dict])

    df['Amount_log'] = np.log1p(df['Amount'])
    df['Hour'] = df['Time'] % (24 * 3600) / 3600.0

    X = df[feature_cols].values

    X_scaled = scaler.transform(X)

    dmatrix = xgb.DMatrix(X_scaled, feature_names=feature_cols)
    proba = bst.predict(dmatrix)[0]

    return {
        'fraud_probability': float(proba),
        'is_fraud': bool(proba >= 0.5),
        'confidence': 'HIGH' if proba > 0.8 or proba < 0.2 else 'MEDIUM'
    }

print("Inference function ready!")

Inference function ready!


In [7]:
import xgboost as xgb

# Test on Real Examples
normal_txn = {'Time': 0.0, 'Amount': 50.0, 'V1': -1.3, 'V2': 1.0, 'V3': 2.5, 'V4': 0.5,
              'V5': -0.5, 'V6': 0.2, 'V7': 0.1, 'V8': 0.3, 'V9': -0.4, 'V10': 0.6,
              'V11': -0.8, 'V12': 1.2, 'V13': -0.3, 'V14': -0.9, 'V15': 0.4,
              'V16': 0.7, 'V17': -0.2, 'V18': 0.5, 'V19': -0.1, 'V20': 0.0,
              'V21': 0.1, 'V22': 0.3, 'V23': -0.2, 'V24': 0.4, 'V25': -0.3,
              'V26': 0.2, 'V27': -0.1, 'V28': 0.05}

fraud_txn = {'Time': 50000.0, 'Amount': 0.0, 'V1': -10.0, 'V2': 8.5, 'V3': -15.0, 'V4': 7.0,
             'V5': -8.0, 'V6': -5.0, 'V7': -10.0, 'V8': 5.0, 'V9': -7.0, 'V10': -10.0,
             'V11': 5.0, 'V12': -12.0, 'V13': -3.0, 'V14': -15.0, 'V15': -2.0,
             'V16': -8.0, 'V17': -12.0, 'V18': -6.0, 'V19': -4.0, 'V20': 2.0,
             'V21': 2.5, 'V22': -1.0, 'V23': -2.0, 'V24': 0.1, 'V25': -1.0,
             'V26': -0.5, 'V27': 3.0, 'V28': 1.5}

print("NORMAL TXN →", predict_fraud(normal_txn))
print("FRAUD TXN  →", predict_fraud(fraud_txn))

NORMAL TXN → {'fraud_probability': 0.0009635353926569223, 'is_fraud': False, 'confidence': 'HIGH'}
FRAUD TXN  → {'fraud_probability': 0.9966821074485779, 'is_fraud': True, 'confidence': 'HIGH'}


In [10]:
final_package = {
    'booster': bst,
    'scaler': scaler,
    'feature_columns': feature_cols,
    'model_info': best_model_data
}

package_path = "/content/drive/MyDrive/Data Science course/Major Projects/Projects/Credit Card Fraud Detection/Credit Card Fraud Detection/models/FINAL_INFERENCE_PACKAGE.pkl"
joblib.dump(final_package, package_path)

print(f"\nFINAL INFERENCE PACKAGE SAVED!")
print(f"→ {package_path}")
print("\nReady finally")


FINAL INFERENCE PACKAGE SAVED!
→ /content/drive/MyDrive/Data Science course/Major Projects/Projects/Credit Card Fraud Detection/Credit Card Fraud Detection/models/FINAL_INFERENCE_PACKAGE.pkl

Ready finally
